In [ ]:
import numpy as np

# Read the data from the cvs file

In [ ]:
data = np.loadtxt('load_data_with_void_markers.csv', 
           dtype={'names': ('point', 'empty', 'x', 'y'), 'formats': 
                  (np.str_, np.str_, np.float_, np.float_)},
                       skiprows=0, delimiter=',', unpack=False)

In [ ]:
data[0]

# Identify the missing indexes

In [ ]:
idx_data_ = np.array([0,1,2,3,5,6,7,8])

In [ ]:
i_grid, j_grid = np.mgrid[0:3, 0:3]

In [ ]:
i_grid_f, j_grid_f = i_grid.flatten(), j_grid.flatten()
i_grid_f_, j_grid_f_ = i_grid_f[idx_data_], j_grid_f[idx_data_]
i_grid_f_, j_grid_f

In [ ]:
P_IJ = np.array([i_grid_f_, j_grid_f_], dtype=np.int_).T
P_IJ

In [ ]:
ij_grid = np.zeros((3,3,), dtype=np.bool_)
ij_grid[i_grid_f_, j_grid_f_] = True
ij_grid

In [ ]:
missing_points = np.where(ij_grid == False)

In [ ]:
mp_i, mp_j = missing_points

In [ ]:
mp_i

In [ ]:
interp_idx_nam = np.array([[mp_i - 1, mp_j],
                       [mp_i + 1, mp_j],
                       [mp_i, mp_j - 1],
                       [mp_i, mp_j + 1]]) 
interp_idx_nam

# Emulate the GOM data with a missing point 

In [ ]:
X, Y = np.mgrid[10:13,20:23] 

In [ ]:
gom_data = np.array([X.flatten(), Y.flatten()]).T
XY_Pa = gom_data
gom_data

In [ ]:
gom_data_ = gom_data[idx_data_]

In [ ]:
gom_data_

# Interpolate the data in the missing points

In [ ]:
X_IJ, Y_IJ = np.zeros((3,3,), dtype=np.float_), np.zeros((3,3,), dtype=np.float_)
X_IJ[i_grid_f_, j_grid_f_] = gom_data_[:,0]
Y_IJ[i_grid_f_, j_grid_f_] = gom_data_[:,1]
X_IJ, Y_IJ

In [ ]:
ij_anm = np.einsum('nam->anm', interp_idx_nam)
i_nm, j_nm = ij_anm

In [ ]:
X_inp = np.average(X_IJ[i_nm, j_nm], axis=-2)
Y_inp = np.average(Y_IJ[i_nm, j_nm], axis=-2)
X_inp, Y_inp

In [ ]:
X_IJ[ij_grid == False] = X_inp
Y_IJ[ij_grid == False] = Y_inp
X_IJ, Y_IJ